In [15]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.externals import joblib
import pickle




In [16]:
df=pd.read_csv('Crop_recommendation.csv')

df.head(10)
#df['label'].unique()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice
5,69,37,42,23.058049,83.370118,7.073454,251.055000,rice
6,69,55,38,22.708838,82.639414,5.700806,271.324860,rice
7,94,53,40,20.277744,82.894086,5.718627,241.974195,rice
8,89,54,38,24.515881,83.535216,6.685346,230.446236,rice
9,68,58,38,23.223974,83.033227,6.336254,221.209196,rice


In [17]:
dummies=pd.get_dummies(df['label'])
df2=pd.concat([df,dummies],axis=1)


In [18]:
mapping=({'rice': 0, 'maize': 1,'chickpea': 2, 'kidneybeans': 3,'pigeonpeas': 4, 'mothbeans': 5,'mungbean': 6, 'blackgram': 7,
           'lentil': 8, 'pomegranate': 9,'banana': 10, 'mango': 11,'grapes': 12, 'watermelon': 13,'muskmelon': 14, 'apple': 15,
           'orange': 16, 'papaya': 17, 'coconut': 18,'cotton': 19, 'jute': 20,'coffee': 21})

mapping = {value : key for (key, value) in mapping.items()}


mapping[18]

'coconut'

In [19]:
x=df.drop('label',axis=1)
y=dummies

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.33)


scaler=MinMaxScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

scaler_filename = "scaler.save"
joblib.dump(scaler, scaler_filename) 


['scaler.save']

In [20]:
model=Sequential()
model.add(Dense(8))
model.add(Dense(800,activation='relu'))
model.add(Dense(800,activation='relu'))
model.add(Dense(800,activation='relu'))
model.add(Dense(22,activation='softmax'))

In [21]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test))

Epoch 1/50
47/47 [==============================] - 2s 28ms/step - loss: 2.0393 - accuracy: 0.3419 - val_loss: 2.2972 - val_accuracy: 0.2383
Epoch 2/50
47/47 [==============================] - 1s 23ms/step - loss: 0.9356 - accuracy: 0.6520 - val_loss: 2.4327 - val_accuracy: 0.3664
Epoch 3/50
47/47 [==============================] - 1s 24ms/step - loss: 0.6794 - accuracy: 0.7537 - val_loss: 1.1845 - val_accuracy: 0.6405
Epoch 4/50
47/47 [==============================] - 1s 23ms/step - loss: 0.5125 - accuracy: 0.8134 - val_loss: 1.6619 - val_accuracy: 0.5000
Epoch 5/50
47/47 [==============================] - 1s 23ms/step - loss: 0.4551 - accuracy: 0.8284 - val_loss: 0.7388 - val_accuracy: 0.7562
Epoch 6/50
47/47 [==============================] - 1s 23ms/step - loss: 0.3637 - accuracy: 0.8541 - val_loss: 1.4802 - val_accuracy: 0.6295
Epoch 7/50
47/47 [==============================] - 1s 23ms/step - loss: 0.3372 - accuracy: 0.8765 - val_loss: 0.3469 - val_accuracy: 0.8719
Epoch 8/50
47

In [22]:
model.save('model.h5')

In [23]:
model.evaluate(x_test,y_test)


23/23 [==============================] - 0s 5ms/step - loss: 0.4217 - accuracy: 0.8843


[0.4217321276664734, 0.8842975497245789]

In [ ]:
predict_x=np.argmin(model.predict(x,verbose=1,batch_size=1,steps=None),axis=1)


c=None

for i in predict_x:
  c=i
  print(mapping[c])

In [26]:
x_train[:1]

array([[0.13571429, 0.82142857, 0.95      , 0.27103274, 0.78005956,
        0.37474606, 0.19176355]])

In [27]:
a=np.array([[0.27857143,0.37142857, 0.07, 0.58501852,0.3693819 ,
        0.45363027,0.52164782]])
a

array([[0.27857143, 0.37142857, 0.07      , 0.58501852, 0.3693819 ,
        0.45363027, 0.52164782]])

In [28]:
x[:1]

,N,P,K,temperature,humidity,ph,rainfall
0,90,42,43,20.879744,82.002744,6.502985,202.935536


In [29]:
input=scaler.transform(x[3:4])
predict_x=model.predict(input) 
classes_x=np.argmin(predict_x,axis=1)

c=None

for i in classes_x:
  c=i

mapping[c]


'muskmelon'

In [30]:
x[0:1]

,N,P,K,temperature,humidity,ph,rainfall
0,90,42,43,20.879744,82.002744,6.502985,202.935536


In [31]:
def crop_prediction(n,p,k,temp,humidity,ph,rainfall):
  a=np.array([[n,p,k,temp,humidity,ph,rainfall]])
  a=scaler.transform(a)
  predict_x=np.argmin(model.predict(a,verbose=0,batch_size=None,steps=None),axis=1)

  
  

  c=None

  for i in predict_x:
    c=i

  return mapping[c]
  





In [32]:
crop_prediction(90,42,43,20,82,6,202)




'muskmelon'

In [33]:
b=np.array([[90,42,43,20,82,6,202]])
scaler.transform(b)




array([[0.64285714, 0.26428571, 0.19      , 0.30788656, 0.7903157 ,
        0.38543803, 0.65346833]])